In [1]:
import os
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision import transforms
import resnet
import torch.nn as nn
import torch.nn.functional as F
from scipy import stats
import gc
import time

/usr/local/home/sgchr/anaconda3/envs/pyt/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/home/sgchr/anaconda3/envs/pyt/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
data_tr = datasets.CIFAR10('data'+ '/CIFAR10', train=True, download=True)
data_te = datasets.CIFAR10('data' + '/CIFAR10', train=False, download=True)
X_tr = data_tr.data
Y_tr = torch.from_numpy(np.array(data_tr.targets))
X_te = data_te.data
Y_te = torch.from_numpy(np.array(data_te.targets))

Files already downloaded and verified
Files already downloaded and verified


In [3]:
num_imp_per_layer = 14000
xt = np.zeros((len(Y_tr), len(np.unique(Y_tr)), num_imp_per_layer))

In [4]:
fisher = torch.zeros(xt.shape[-1], xt.shape[-1],dtype=torch.double).cuda()
iterates = torch.zeros(xt.shape[-1], xt.shape[-1], dtype=torch.double)

In [1]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [6]:
import time

# A List of Items
items = list(range(0, 3))
l = len(items)

# Initial call to print 0% progress
print("test")
printProgressBar(0, l, prefix = 'Progress1:', suffix = 'Complete', length = 50)
for i, item in enumerate(items):
    # Do stuff...
    time.sleep(0.1)
    # Update Progress Bar
    #printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
    printProgressBar(0, l, prefix = 'Progress2:', suffix = 'Complete', length = 50)
    for j, item in enumerate(items):
        # Do stuff...
        time.sleep(0.1)
        # Update Progress Bar
        printProgressBar(j + 1, l, prefix = 'Progress2:', suffix = 'Complete', length = 50)
    printProgressBar(i + 1, l, prefix = 'Progress1:', suffix = 'Complete', length = 50)

test
Progress2: |██████████████████████████████████████████████████| 100.0% Complete
Progress2: |██████████████████████████████████████████████████| 100.0% Complete
Progress2: |██████████████████████████████████████████████████| 100.0% Complete
Progress1: |██████████████████████████████████████████████████| 100.0% Complete


In [7]:

def select(X, K, fisher, iterates, lamb=1, backwardSteps=0, nLabeled=0):
    '''
    K is the number of images to be selected for labelling, 
    iterates is the fisher for images that are already labelled
    '''

    numEmbs = len(X)
    dim = X.shape[-1]
    rank = X.shape[-2]
    indsAll = []

    currentInv = torch.inverse(lamb * torch.eye(dim).cuda() + iterates.cuda() * nLabeled / (nLabeled + K))
    # what is lamb used for here?
    #X = X * np.sqrt(K / (nLabeled + K))
    fisher = fisher.cuda()
    total = 0
    rounds = int((backwardSteps + 1) *  K)
    printProgressBar(0, rounds, prefix = 'First Loop Progress:', suffix = 'Complete', length = 50)
    # forward selection
    for i in range(rounds): 
        #print("Start outer loop iteration ", i)
        '''
        K corresponds to minibatch size, which is called B in the paper.
        currently we assume that backwardSteps = 0
        '''

        # xt_ = X.cuda()
        xt_ = X  
        '''
        in the calculation below, traceEst has X.shape[0] elements.
        The calculation done for computing one element of traceEst
        has no effect on the calculation done for computing other
        elements of traceEst. This suggests that we can compute  
        tracEst in chunks, rather than computing all elements in 
        one go.

        traceEst = torch.zeros(X.shape[0])
        chunkSize = 100
        for c_idx in range(0, X.shape[0], chunkSize):
            xt_chunk = xt_[c_idx * chunkSize : (c_idx + 1) * chunkSize]
            innerInv = torch.inverse(torch.eye(rank).cpu() + xt_chunk @ currentInv @ xt_chunk.transpose(1, 2)).detach()
            traceEst[c_idx * chunkSize : (c_idx + 1) * chunkSize] = torch.diagonal(
                xt_chunk @ currentInv @ fisher @ currentInv @ xt_chunk.transpose(1, 2) @ innerInv, 
                dim1=-2, 
                dim2=-1
            ).sum(-1) 
        '''


        # innerInv = torch.inverse(torch.eye(rank).cuda() + xt_ @ currentInv @ xt_.transpose(1, 2)).detach()
        # innerInv[torch.where(torch.isinf(innerInv))] = torch.sign(innerInv[torch.where(torch.isinf(innerInv))]) * np.finfo('float32').max
        
        
        
        # traceEst = torch.diagonal(
        #     xt_ @ currentInv @ fisher @ currentInv @ xt_.transpose(1, 2) @ innerInv, 
        #     dim1=-2, 
        #     dim2=-1
        # ).sum(-1)
        traceEst = np.zeros(X.shape[0]) #torch.zeros(X.shape[0]).cuda() 
        chunkSize = 2500
        #print(X.shape[0])
        
        time_for_inner_loop = time.time()
        for c_idx in range(0, X.shape[0], chunkSize):
            xt_chunk = xt_[c_idx * chunkSize : (c_idx + 1) * chunkSize]
            xt_chunk = torch.tensor(xt_chunk).cuda() #.clone().detach()
            innerInv = torch.inverse(torch.eye(rank).cuda() + xt_chunk @ currentInv @ xt_chunk.transpose(1, 2))
            innerInv[torch.where(torch.isinf(innerInv))] = torch.sign(innerInv[torch.where(torch.isinf(innerInv))]) * np.finfo('float32').max
            traceEst[c_idx * chunkSize : (c_idx + 1) * chunkSize] = torch.diagonal(
                xt_chunk @ currentInv @ fisher @ currentInv @ xt_chunk.transpose(1, 2) @ innerInv, 
                dim1=-2, 
                dim2=-1
            ).sum(-1).detach().cpu()
        time_for_inner_loop_end = time.time()
        #print('inner loop time: ', time_for_inner_loop_end-time_for_inner_loop)
        total += (time_for_inner_loop_end-time_for_inner_loop)
        '''
        Vx^T M^-1 I(θ_L) M^-1 Vx A^-1 formula from page 5 of paper.
        currentInv corresponds to M^-1
        fisher corresponds to I(θ_L)
        xt_ corresponds to Vx^T
        innerInv corresponds to A^-1
        '''

        xt = xt_
        del xt, innerInv
        #del xt_, innerInv
        torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.empty_cache()
        gc.collect()

        # traceEst = traceEst.detach().cpu().numpy() # objective value in eq (5) from the paper

        dist = traceEst - np.min(traceEst) + 1e-10
        dist = dist / np.sum(dist)
        sampler = stats.rv_discrete(values=(np.arange(len(dist)), dist))
        ind = sampler.rvs(size=1)[0]
        for j in np.argsort(dist)[::-1]:
            if j not in indsAll:
                ind = j
                break

        indsAll.append(ind)  # adding a new tilde_x to the minibatch being made
        #print(i, ind, traceEst[ind], flush=True)
       
        xt_ = torch.tensor(X[ind]).unsqueeze(0).cuda()
        innerInv = torch.inverse(torch.eye(rank).cuda() + xt_ @ currentInv @ xt_.transpose(1, 2)).detach()
        currentInv = (currentInv - currentInv @ xt_.transpose(1, 2) @ innerInv @ xt_ @ currentInv).detach()[0]
        printProgressBar(i + 1, rounds, prefix = 'First Loop Progress:', suffix = 'Complete', length = 50)

    print("Average time for inner for loop of select function: ", (total/int((backwardSteps + 1) *  K)))
    # backward pruning
    second_for_loop_time = time.time()
    rounds = len(indsAll) - K
    printProgressBar(0, rounds, prefix = 'Second Loop Progress:', suffix = 'Complete', length = 50)
    for i in range(rounds):

        # select index for removal
        xt_ = torch.tensor(X[indsAll]).cuda()
        innerInv = torch.inverse(-1 * torch.eye(rank).cuda() + xt_ @ currentInv @ xt_.transpose(1, 2)).detach()
        traceEst = torch.diagonal(xt_ @ currentInv @ fisher @ currentInv @ xt_.transpose(1, 2) @ innerInv, dim1=-2, dim2=-1).sum(-1)
        delInd = torch.argmin(-1 * traceEst).item()
        #print(i, indsAll[delInd], -1 * traceEst[delInd].item(), flush=True)


        # compute new inverse
        xt_ = torch.tensor(X[indsAll[delInd]]).unsqueeze(0).cuda()
        innerInv = torch.inverse(-1 * torch.eye(rank).cuda() + xt_ @ currentInv @ xt_.transpose(1, 2)).detach()
        currentInv = (currentInv - currentInv @ xt_.transpose(1, 2) @ innerInv @ xt_ @ currentInv).detach()[0]

        del indsAll[delInd]
        printProgressBar(i + 1, rounds, prefix = 'Second Loop Progress:', suffix = 'Complete', length = 50)
    second_for_loop_time_end = time.time()
    print("The second for loop in the select function took ", (second_for_loop_time_end-second_for_loop_time))
    del xt_, innerInv, currentInv
    torch.cuda.empty_cache()
    gc.collect()
    return indsAll

In [8]:
n_pool = len(Y_tr)
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_unlabeled = np.arange(n_pool)[~idxs_lb]

In [9]:
chosen = select(xt[idxs_unlabeled], 1000, fisher, iterates, lamb = 1, backwardSteps = 1, nLabeled=np.sum(idxs_lb))

In [ ]:
class DataHandler3(Dataset):
    def __init__(self, X, Y, transform= None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.X[index], self.Y[index]
        if self.transform is not None:
            x = Image.fromarray(x)
            x = self.transform(x)
        return x, y, index
    def __len__(self):
        return len(self.Y)

In [ ]:
def get_handler(name):
    if name == 'CIFAR10':
        return DataHandler3

In [ ]:
handler = get_handler('CIFAR10')

In [ ]:
print(type(get_handler))

In [ ]:
args={'n_epoch': 3, 'transform': transforms.Compose([ 
                     transforms.RandomCrop(32, padding=4),
                     transforms.RandomHorizontalFlip(),
                     transforms.ToTensor(),
                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
                 ]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 1},
                 'loader_te_args':{'batch_size': 1000, 'num_workers': 1}, # change back to 1000
                 'optimizer_args':{'lr': 0.05, 'momentum': 0.3},
                 'transformTest': transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])}

In [ ]:
def predict_prob(X, Y, model, exp=True):
    # if type(model) == list: model = clf

    # loader_te = DataLoader(handler(X, Y, transform=args['transformTest']), shuffle=False, **args['loader_te_args'])
    # print(type(loader_te))
    loader_te = DataLoader(X_tr, batch_size = 100, shuffle = False)
    model = model.eval()
    probs = torch.zeros([len(Y), len(np.unique(Y))])
    with torch.no_grad():
        for idx, x in enumerate(loader_te):
            x = Variable(x.cuda())
            out, e1 = model(x)
            if exp: out = F.softmax(out, dim=1)
            probs[idx] = out.cpu().data
            # probs = out.cpu().data
    
    return probs

In [ ]:
innerInv = torch.tensor(())

In [ ]:
X_tr[:100].shape[0]

In [ ]:
loader_te = DataLoader(X_tr, batch_size = 100, shuffle = False)
for idx,x in enumerate(loader_te):
    print(x.shape)

In [ ]:
model = resnet.ResNet18()

In [ ]:
phat = predict_prob(X_tr[idxs_unlabeled], Y_tr[idxs_unlabeled],model)

In [ ]:
print('all probs: ' + 
                str(str(torch.mean(torch.max(phat, 1)[0]).item())) + ' ' + 
                str(str(torch.mean(torch.min(phat, 1)[0]).item())) + ' ' + 
                str(str(torch.mean(torch.std(phat,1)).item())), flush=True)

In [1]:
import pickle
savefile = open("defaultsavefilename.p", "br")
save = pickle.load(savefile)
acc_dict = save
print(acc_dict)

{'FISH': [0.0999], 'BAIT': [0.1002, 0.1005]}
